In [24]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, html, datetime 
from os import listdir
from os.path import isfile, join
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender 
from lib.script_generator import script_generator   
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')

In [25]:
def normalize(s): 
    s=re.sub('[^A-Za-z0-9\s]','',str(s))
    s=re.sub('\s{1,}','',str(s))
    return s.upper().strip()
normalize('Jump host access (e.g., the user accesses the HVA from an intermediate host) ')

'JUMPHOSTACCESSEGTHEUSERACCESSESTHEHVAFROMANINTERMEDIATEHOST'

In [26]:
pl=sql_todf("""
SELECT UPPER(LTRIM(RTRIM(DisplayValue))) [Selections], PK_PickList [ML_Value], PK_PickListType [PLT]
FROM vwPickLists  
WHERE PK_PickListType > 410
ORDER BY PK_PickListType DESC
""", connstr)  
s=list(pl.loc[pl['PLT']==491]['Selections'])
pl['NSelections'] = pl['Selections'].apply(normalize) 
pr = picklist_recommender(ctx.config['connstr'], reset_cache=True, picklist_where=' PK_PickListType > 410 ') 
#pr.normalize(' '.join(s))
pl.loc[pl['PLT']==491] 

,Selections,ML_Value,PLT,NSelections
34,SECURE ACCESS SERVICES EDGE (SASE),5460,491,SECUREACCESSSERVICESEDGESASE
35,"SECURE WEB GATEWAY (E.G., CITRIX OR VMWARE HOR...",5461,491,SECUREWEBGATEWAYEGCITRIXORVMWAREHORIZON
36,"JUMP HOST ACCESS (E.G., THE USER ACCESSES THE ...",5462,491,JUMPHOSTACCESSEGTHEUSERACCESSESTHEHVAFROMANINT...
37,"DIRECT VPN ACCESS (E.G., THE USER ESTABLISHES ...",5463,491,DIRECTVPNACCESSEGTHEUSERESTABLISHESAVPNCONNECT...
38,"INDIRECT VPN ACCESS (E.G., THE USER ESTABLISHE...",5464,491,INDIRECTVPNACCESSEGTHEUSERESTABLISHESAVPNCONNE...
39,"REVERSE PROXY ACCESS (E.G., THE USER ESTABLISH...",5465,491,REVERSEPROXYACCESSEGTHEUSERESTABLISHESACONNECT...
40,"DIAL-UP ACCESS (E.G., THE USER ESTABLISHES A M...",5466,491,DIALUPACCESSEGTHEUSERESTABLISHESAMODEMCONNECTI...
41,"DIRECT ACCESS VIA THE INTERNET (E.G., THE USER...",5467,491,DIRECTACCESSVIATHEINTERNETEGTHEUSERESTABLISHES...
42,OTHER,5468,491,OTHER
43,UNKNOWN,5469,491,UNKNOWN


In [27]:
path=r'C:\Users\timko\Downloads\Questionnaire\\'
files = [f for f in listdir(path) if re.search('^HVA.*1\.xlsx', f)]
print('\n'.join(files))
xlfile=files[7]
metric='EP7a' 
xlfile  

HVA Asset Management Questionnaire v0.1.xlsx
HVA Business Continuity Management Questionnaire v0.1.xlsx
HVA Data Protection Questionnaire v0.1.xlsx
HVA Endpoint Protection Questionnaire v0.1.xlsx
HVA Identity and Access Management Questionnaire v0.1.xlsx
HVA Monitoring and Detection Questionnaire v0.1.xlsx
HVA Network Architecture Questionnaire v0.1.xlsx
HVA Remote Access Questionnaire v0.1.xlsx
HVA Vulnerability Management Questionnaire v0.1.xlsx


'HVA Remote Access Questionnaire v0.1.xlsx'

In [28]:
converts={
    'Selections':lambda x: re.sub('^(\s?\d{,2}\.)','',str(x)).upper().strip()
    , 'ML':lambda x: str(x).strip()
    , 'SML':lambda x: re.sub('[^A-Z0-9]','',str(x)).strip()  }
df=pd.read_excel(f'{path}{xlfile}', converters=converts,  sheet_name=4, header=2, usecols='B:I').reset_index()
df = df.rename(columns={'ML Score': 'ML_Score','ML Scored': 'ML_Scored' })
#df.drop(index=df.index[-15:], inplace=True)
for c in ['ID','Question','SML']:
    df[c] = df[c].replace(to_replace=[0, '0'], method='ffill')
if not 'ML' in df.columns:
    df['ML']=df["ML_Score"].apply(lambda x: str(x).strip())
    df['ML_Score']=df["ML_Scored"]
 
df["ID"].fillna(method='ffill', inplace=True) 
df.drop(columns=['Check Answer', 'index'], inplace=True)

df = df.loc[ df["ID"].str.contains('^\w\w\d(\w){0,5}$', na=False, regex=True) ]  
df = df.loc[ df["ML"].str.contains('\d', na=False,  regex=True, case=False) ]  
df['ML_Score'] = df['ML_Score'].apply(lambda s: re.sub('[^0-9\.]','',str(s)))
df['NSelections'] = df['Selections'].apply(normalize)
df.loc[df['ID']=='RA2b']['NSelections']
 

12                         SECUREACCESSSERVICESEDGESASE
13              SECUREWEBGATEWAYEGCITRIXORVMWAREHORIZON
14    JUMPHOSTACCESSEGTHEUSERACCESSESTHEHVAFROMANINT...
15    DIRECTVPNACCESSEGTHEUSERESTABLISHESAVPNCONNECT...
16    INDIRECTVPNACCESSEGTHEUSERESTABLISHESAVPNCONNE...
17    REVERSEPROXYACCESSEGTHEUSERESTABLISHESACONNECT...
18    DIALUPACCESSEGTHEUSERESTABLISHESAMODEMCONNECTI...
19    DIRECTACCESSVIATHEINTERNETEGTHEUSERESTABLISHES...
20                                                OTHER
21                                              UNKNOWN
Name: NSelections, dtype: object

In [29]:
s= list(df.loc[df['ID']=='RA2b']['NSelections'])
pr.recommend(s, normalizer=normalize)['PK_PicklistType'] 


491

In [30]:
dfp=df.groupby(['ID'], as_index=False).agg({'NSelections':list})  
dfp['PLT']=dfp['NSelections'].apply(lambda s: pr.recommend(s, normalizer=normalize)['PK_PicklistType'])
pr.to_cache() 

dff=pd.merge(df,dfp.loc[:,['ID','PLT']], how='left', left_on='ID', right_on='ID')
dff=pd.merge(dff,pl, how='left', left_on=['NSelections','PLT'], right_on=['NSelections','PLT'])
dff.ML_Value=dff.ML_Value.fillna(0).astype('int32').astype('str') 
dff

,ID,Question,Selections_x,SML,ML,ML_Score,ML_Scored,NSelections,PLT,Selections_y,ML_Value
0,RA2b,If Yes to RA2: What type(s) of remote user acc...,SECURE ACCESS SERVICES EDGE (SASE),SML3,3,1,0,SECUREACCESSSERVICESEDGESASE,491,SECURE ACCESS SERVICES EDGE (SASE),5460
1,RA2b,If Yes to RA2: What type(s) of remote user acc...,"SECURE WEB GATEWAY (E.G., CITRIX OR VMWARE HOR...",SML3,3,1,0,SECUREWEBGATEWAYEGCITRIXORVMWAREHORIZON,491,"SECURE WEB GATEWAY (E.G., CITRIX OR VMWARE HOR...",5461
2,RA2b,If Yes to RA2: What type(s) of remote user acc...,"JUMP HOST ACCESS (E.G., THE USER ACCESSES THE ...",SML2,2,1,0,JUMPHOSTACCESSEGTHEUSERACCESSESTHEHVAFROMANINT...,491,"JUMP HOST ACCESS (E.G., THE USER ACCESSES THE ...",5462
3,RA2b,If Yes to RA2: What type(s) of remote user acc...,"DIRECT VPN ACCESS (E.G., THE USER ESTABLISHES ...",SML2,2,1,0,DIRECTVPNACCESSEGTHEUSERESTABLISHESAVPNCONNECT...,491,"DIRECT VPN ACCESS (E.G., THE USER ESTABLISHES ...",5463
4,RA2b,If Yes to RA2: What type(s) of remote user acc...,"INDIRECT VPN ACCESS (E.G., THE USER ESTABLISHE...",SML2,2,1,0,INDIRECTVPNACCESSEGTHEUSERESTABLISHESAVPNCONNE...,491,"INDIRECT VPN ACCESS (E.G., THE USER ESTABLISHE...",5464
...,...,...,...,...,...,...,...,...,...,...,...
148,RA8b5,If Yes to RA8b: Are these connections encrypted?,NO,SML1,1,0,0,NO,423,NO,4715
149,RA8b5,If Yes to RA8b: Are these connections encrypted?,UNKNOWN,SML1,1,0.001,0,UNKNOWN,423,UNKNOWN,4716
150,RA8b6,If Yes to RA8b: Are these connections logged a...,YES,SML2,2,1,0,YES,423,YES,4714
151,RA8b6,If Yes to RA8b: Are these connections logged a...,NO,SML2,2,0,0,NO,423,NO,4715


In [31]:
df1=dff
df1.SML.fillna(method='ffill', inplace=True)
df1=df1.loc[df1['ML_Score'] != '0']
df1['sortorder'] = range(1, len(df1)+1)
df1=df1.loc[:, ['ID', 'SML', 'ML','ML_Score', 'ML_Scored', 'ML_Value', 'sortorder']]
sql=SQL_INSERT_FROM_DF(df1, TARGET='@SASCORING')
with open(f'{ctx.get_dest()}script.sql', 'w') as f:
    f.write('\n'.join(sql))
df1.loc[df1['ID']=='RA2b']

--DECLARE @SASCORING AS TABLE (ID NVARCHAR(MAX), SML NVARCHAR(MAX), ML NVARCHAR(MAX), ML_Score NVARCHAR(MAX), ML_Scored NVARCHAR(MAX), ML_Value NVARCHAR(MAX), sortorder INT)


,ID,SML,ML,ML_Score,ML_Scored,ML_Value,sortorder
0,RA2b,SML3,3,1,0,5460,1
1,RA2b,SML3,3,1,0,5461,2
2,RA2b,SML2,2,1,0,5462,3
3,RA2b,SML2,2,1,0,5463,4
4,RA2b,SML2,2,1,0,5464,5
5,RA2b,SML2,2,1,0,5465,6
6,RA2b,SML1,1,1,0,5466,7
7,RA2b,SML1,1,1,0,5467,8
9,RA2b,SML1,1,0.001,0,5469,9


In [23]:
def writefile():
    s=''
    template="2023-A-HVAENDPRO'\n\tBEGIN\n\n\tEND--'2023-A-HVAENDPRO'"
    with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Sprocs\sp_HVASAScoreDash.sql', 'r', encoding='UTF-8') as f:
        s=f.read()
    m=re.search("(2023-A-HVAENDPRO.*--'2023-A-HVAENDPRO')",s, re.DOTALL)
    pholder=s[m.span(1)[0] :m.span(1)[1] ]
    s=s.replace(pholder, '')
    s
def check_pick(id='EP7a'):
    sel = list(df.loc[df['ID']==id]['Selections'])
    print(' '.join(sel))
    pr.recommend(sel, normalizer=normalize)